# Troubleshooting Agent chat for an online customer service scenario

This notebook uses the following agents:

1. __User proxy__: Represents the human user who provides an initial complaint to customer service.
2. __Troubleshooter__: Troubleshooting agent responsible for analyzing the data from various sources to identify the potential problem. Troubleshooter uses general guidance on problem resolution, but needs to have the agency and capability of findng the solution amidst confusing and conflicting information.
3. __Data engineer__: Retrieves the needed data from the correct data sources to enable the troubleshooting process

In [1]:
from IPython import get_ipython
from typing_extensions import Annotated
from dotenv import load_dotenv, find_dotenv

import autogen
from autogen.cache import Cache
import os

load_dotenv(find_dotenv())

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST.json",
    filter_dict={
        "model": ["gpt-4o", "gpt4-turbo"],
    },
)

settings = {
    'host': os.getenv('ACCOUNT_HOST'),
    'master_key': os.getenv('ACCOUNT_KEY'),
    'database_id': os.getenv('COSMOS_DATABASE'),
    'container_id': os.getenv('COSMOS_CONTAINER'),
}

for key, value in settings.items():
    if value is None:
        raise ValueError(f"Missing environment variable for {key}")

In [2]:
troubleshooter_prompt = """
You are an experienced telecommunications troubleshooting expert working alongside a data engineer. Your task is to analyze and resolve user-reported issues with their telco systems. Follow these steps:
Carefully review the user's question or problem description.
Request relevant data from the data engineer, including customer information and mass outage details.
Analyze the provided data, paying close attention to:
1. Complaint history
2. Status of connection (if not "active," consider non-payment or pending fixes)
3. Any ongoing mass outages in the area

To retrieve customer information, you must pass in a customer id. 
To retrieve mass outage info, you must pass in the address of the customer which you can only get by retrieving customer information first. 

Develop a comprehensive solution based on your analysis.
Present a clear, concise, step-by-step resolution to the user, including any necessary assumptions.
If additional information is required, ask the user specific questions to gather more details.
"""

data_engineer_prompt = """
You are a data engineer, working with a master troubleshooter. You can retrieve the user data based on the customer id (function get_customer_info) and you can retrieve the mass outage information(function get_outage_info). 
"""

In [3]:
import tempfile

from autogen import ConversableAgent
from autogen.coding import DockerCommandLineCodeExecutor

llm_config = {"config_list": config_list, "cache_seed": 42}
user_proxy = autogen.UserProxyAgent(
    name="User_proxy",
    system_message="A human admin.",
    human_input_mode="ALWAYS",
    code_execution_config=False
)
troubleshooter = autogen.ConversableAgent(
    name="troubleshooter",
    system_message= troubleshooter_prompt,
    llm_config=llm_config,
)

temp_dir = tempfile.TemporaryDirectory()

executor = DockerCommandLineCodeExecutor(
    image="python:3.12-slim", 
    timeout=10, 
    work_dir=temp_dir.name,  
)
data_engineer = autogen.AssistantAgent(
    name="data_engineer",
    system_message=data_engineer_prompt,
    llm_config=llm_config,
    code_execution_config={"executor": executor},
    human_input_mode="NEVER"
)


groupchat = autogen.GroupChat(agents=[user_proxy, troubleshooter, data_engineer], messages=[], max_round=50)
manager = autogen.GroupChatManager(groupchat=groupchat, llm_config=llm_config)


In [4]:
import json
import jsonlines
import requests
import azure.cosmos.documents as documents
import azure.cosmos.cosmos_client as cosmos_client
import azure.cosmos.exceptions as exceptions
from azure.cosmos.partition_key import PartitionKey
import json

HOST = settings['host']
MASTER_KEY = settings['master_key']
DATABASE_ID = settings['database_id']
CONTAINER_ID = settings['container_id']

client = cosmos_client.CosmosClient(HOST, {'masterKey': MASTER_KEY})
db = client.get_database_client(DATABASE_ID)
container = db.get_container_client(CONTAINER_ID)

@data_engineer.register_for_execution()
@troubleshooter.register_for_llm(name="get_customer_information", description="Retrieve customer data from the database")
def get_customer_info(customer_id: str) -> str:
    try:
        query = f"SELECT * FROM c WHERE c.id = '{customer_id}'"
        items = list(container.query_items(query=query, enable_cross_partition_query=True))
        
        if items:
            customer_info = items[0]
            return json.dumps(customer_info, default=str)
        else:
            error_message = {'error': f"Customer with ID '{customer_id}' not found."}
            return json.dumps(error_message)
    
    except exceptions.CosmosHttpResponseError as e:
        error_message = {'error': str(e)}
        return json.dumps(error_message)


  

# In our case a dummy outage.jsonl file is used to simulate the outage information
@data_engineer.register_for_execution()  
@troubleshooter.register_for_llm(name="get_outage_info", description="Retrieve the mass outage information for a customer's city")  
def get_outage_info(customer_address: str) -> str:  
    city = customer_address.split(', ')[1]  
      
    try:  
        with jsonlines.open('outage.jsonl') as reader:  
            for obj in reader:  
                if obj['city'] == city:  
                    return json.dumps(obj)  
    except FileNotFoundError:  
        return json.dumps({  
            'status': 'error',  
            'message': 'Outage information file not found.',  
            'affected_services': [],  
            'estimated_resolution_time': None  
        })  
      
    default_message = {  
        'status': 'unknown',  
        'message': f"No outage information available for {city}.",  
        'affected_services': [],  
        'estimated_resolution_time': None  
    }  
    return json.dumps(default_message)  



In [6]:
user_proxy.initiate_chat(
    manager, message="I'm experiencing issues with my internet connection. My customer id is Customer5. Can you help me troubleshoot?"
)


User_proxy (to chat_manager):

I'm experiencing issues with my internet connection. My customer id is Customer5. Can you help me troubleshoot?

--------------------------------------------------------------------------------

>>>>>>>> USING AUTO REPLY...
troubleshooter (to chat_manager):

***** Suggested tool call (call_MdTg2e9yJhSjzQAZJXkavdiE): get_customer_information *****
Arguments: 
{"customer_id":"Customer5"}
*****************************************************************************************

--------------------------------------------------------------------------------

>>>>>>>> EXECUTING FUNCTION get_customer_information...
data_engineer (to chat_manager):

data_engineer (to chat_manager):

***** Response from calling tool (call_MdTg2e9yJhSjzQAZJXkavdiE) *****
{"id": "Customer5", "FullName": "Sophia Taylor", "Age": 61, "Address": "288 T Street, Munich", "ServiceTier": "Standard", "InternetConnectionType": "Cable", "ContractTerm": 24, "MonthlyBill": 199.94, "PhoneNumber

TypeError: ChatResult(chat_id=None, chat_history=[{'content': "I'm experiencing issues with my internet connection. My customer id is Customer5. Can you help me troubleshoot?", 'role': 'assistant'}, {'content': '', 'tool_calls': [{'id': 'call_MdTg2e9yJhSjzQAZJXkavdiE', 'function': {'arguments': '{"customer_id":"Customer5"}', 'name': 'get_customer_information'}, 'type': 'function'}], 'name': 'troubleshooter', 'role': 'assistant'}, {'content': '{"id": "Customer5", "FullName": "Sophia Taylor", "Age": 61, "Address": "288 T Street, Munich", "ServiceTier": "Standard", "InternetConnectionType": "Cable", "ContractTerm": 24, "MonthlyBill": 199.94, "PhoneNumber": "+19948267530", "Email": "sophia.taylor@email3.com", "ConnectionStatus": "Pending", "ComplaintHistory": "Last 6 months", "_rid": "ouUXALKeqZ8tAAAAAAAAAA==", "_self": "dbs/ouUXAA==/colls/ouUXALKeqZ8=/docs/ouUXALKeqZ8tAAAAAAAAAA==/", "_etag": "\\"b900b601-0000-4700-0000-668fca950000\\"", "_attachments": "attachments/", "_ts": 1720699541}'